In [ ]:
import albumentations as A
img_trans = A.Compose([
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=(0, 0.2), contrast_limit=(0, 0.5), p=0.8),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.Sharpen(alpha=(0.2, 0.5), lightness=(0.2, 0.5), always_apply=False, p=1.0),
            A.ColorJitter(brightness = 0.2, saturation = 0.5, hue = 0.5),
            A.VerticalFlip(),
            A.HorizontalFlip(),
            ])
            
img_trans()

In [5]:
from datetime import datetime
date = datetime.now().strftime("%d%m%Y%H%M%S")
print(date)
datef = datetime.now().strftime("%d-%m-%y,%H:%M:%S")
print(datef)

05112022164747
05-11-22,16:47:47


In [2]:
x = [1,2,3]
import numpy as np
print(np.mean(x))

2.0


In [11]:
import torch
import timm.models as models
from timm import create_model
model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))
# print(model_names)
x = torch.randn((16,3,600,600))
model = create_model("efficientnet_b6", pretrained=False)
# print(model)
# model = model.cuda()
output = model(x)
print(output.shape)
print("total params : {:,}".format(sum([p.data.nelement() for p in model.parameters()])))

torch.Size([16, 1000])
total params : 43,040,704


In [34]:
import torch
import torchvision.models as models
import warnings
warnings.filterwarnings("ignore")

model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))
# print(model_names)

x = torch.randn((16,3,512,512))

model = models.resnet50(pretrained=True)
output = model(x)
print(output.shape)
print("total params : {:,}".format(sum([p.data.nelement() for p in model.parameters()])))

# model = models.efficientnet_b0(pretrained=True)
# output = model(x)
# print(output.shape)
# print("total params : {:,}".format(sum([p.data.nelement() for p in model.parameters()])))

torch.Size([16, 1000])
total params : 25,557,032


In [18]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = "3,4"

import wandb
from datetime import datetime
temp = datetime.now().strftime("%d-%m-%y,%H:%M:%S")
# wandb.init(project="Cencer-Metastasis", entity="jaejungscene", name=temp)
del temp


import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

CFG = {
    'IMG_SIZE':512,
    'EPOCHS':5,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정




train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')




train_df['암의 장경'] = train_df['암의 장경'].fillna(train_df['암의 장경'].mean())
train_df = train_df.fillna(0)

test_df['암의 장경'] = test_df['암의 장경'].fillna(train_df['암의 장경'].mean())
test_df = test_df.fillna(0)



train_df, val_df, train_labels, val_labels = train_test_split(
                                                    train_df.drop(columns=['N_category']), 
                                                    train_df['N_category'], 
                                                    test_size=0.2, 
                                                    random_state=CFG['SEED']
                                                )


def get_values(value):
    return value.values.reshape(-1, 1)

numeric_cols = ['나이', '암의 장경', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2_SISH_ratio']
ignore_cols = ['ID', 'img_path', 'mask_path', '수술연월일', 'N_category']

for col in train_df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        scaler = StandardScaler()
        train_df[col] = scaler.fit_transform(get_values(train_df[col]))
        val_df[col] = scaler.transform(get_values(val_df[col]))
        test_df[col] = scaler.transform(get_values(test_df[col]))
    else:
        le = LabelEncoder()
        train_df[col] = le.fit_transform(get_values(train_df[col]))
        val_df[col] = le.transform(get_values(val_df[col]))
        test_df[col] = le.transform(get_values(test_df[col]))

class CustomDataset(Dataset):
    def __init__(self, medical_df, labels, transforms=None, test=False):
        self.medical_df = medical_df
        self.transforms = transforms
        self.labels = labels
        self.test = test
        
    def __getitem__(self, index):
        if self.test:
            data_path = "/home/ljj0512/private/workspace/CP_Multi-modal_Cencer-metastasis_DACON/data/test_imgs"
        else:
            data_path = "/home/ljj0512/private/workspace/CP_Multi-modal_Cencer-metastasis_DACON/data/train_imgs"
        img_path = os.path.join(data_path,self.medical_df["img_path"].iloc[index][-14:])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
                
        if self.labels is not None:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', 'mask_path', '수술연월일']).iloc[index])
            label = self.labels[index]
            return image, tabular, label
        else:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', '수술연월일']).iloc[index])
            return image, tabular
        
    def __len__(self):
        return len(self.medical_df)


train_transforms = A.Compose([
                            A.HorizontalFlip(),
                            A.VerticalFlip(),
                            A.Rotate(limit=90, border_mode=cv2.BORDER_CONSTANT,p=0.3),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transforms = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])


train_dataset = CustomDataset(train_df, train_labels.values, train_transforms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
print(train_dataset[0][0].shape)
val_dataset = CustomDataset(val_df, val_labels.values, test_transforms)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

torch.Size([3, 512, 512])
